<a href="https://colab.research.google.com/github/AnuragSingh7292/Remote-sensing-image-retrieval-deep-learning-/blob/main/ResNetModel%26CombineFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# After running, follow the authentication steps


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define the paths
zip_path = "/content/drive/MyDrive/Images.zip"  # Update with your actual file path
extract_path = "/content/UC_Merced_LandUse"  # Temporary extraction path

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Extracted categories:", os.listdir(extract_path))  # Should list 21 folders


Extracted categories: ['Images']


ResNet50

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=True)

# Extracting features from 'conv5_block3_out' (Last convolutional block)
conv5_layer = tf.keras.Model(inputs=base_model.input,
                             outputs=base_model.get_layer('conv5_block3_out').output)

# Extracting features from 'avg_pool' (Instead of FC6, since ResNet50 does not have fully connected layers)
fc6_layer = tf.keras.Model(inputs=base_model.input,
                           outputs=base_model.get_layer('avg_pool').output)

# Function to extract features
def extract_features(model, img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
conv5_features, fc6_features, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_feat = extract_features(conv5_layer, img_path)
                fc6_feat = extract_features(fc6_layer, img_path)
                conv5_features.append(conv5_feat)
                fc6_features.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
conv5_features = np.array(conv5_features)
fc6_features = np.array(fc6_features)

# Apply PCA to reduce dimensions to (2100, 32)
pca_conv5 = PCA(n_components=32)
conv5_reduced = pca_conv5.fit_transform(conv5_features)

pca_fc6 = PCA(n_components=32)
fc6_reduced = pca_fc6.fit_transform(fc6_features)

# Save features as separate .npy files
np.save(os.path.join(save_dir, "resnet50_conv5_32_features.npy"), conv5_reduced)
np.save(os.path.join(save_dir, "resnet50_fc6_32_features.npy"), fc6_reduced)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted and saved successfully for ResNet50! (Shape: (2100, 32))")


Resnet101

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'conv5_block3_out' (Last convolutional block)
conv5_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                                 outputs=base_model_101.get_layer('conv5_block3_out').output)

# Extracting features from 'avg_pool'
fc6_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

# Function to extract features
def extract_features(model, img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
conv5_features_101, fc6_features_101, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_feat = extract_features(conv5_layer_101, img_path)
                fc6_feat = extract_features(fc6_layer_101, img_path)
                conv5_features_101.append(conv5_feat)
                fc6_features_101.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
conv5_features_101 = np.array(conv5_features_101)
fc6_features_101 = np.array(fc6_features_101)

# Save features as separate .npy files
np.save(os.path.join(save_dir, "resnet101_conv5_con5_features.npy"), conv5_features_101)
np.save(os.path.join(save_dir, "resnet101_fc6_fc6_features.npy"),fc6_features_101)

# Apply PCA to reduce dimensions to (2100, 32)
pca_conv5_101 = PCA(n_components=32)
conv5_reduced_101 = pca_conv5_101.fit_transform(conv5_features_101)

pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save features as separate .npy files
np.save(os.path.join(save_dir, "resnet101_conv5_32_features.npy"), conv5_reduced_101)
np.save(os.path.join(save_dir, "resnet101_fc6_32_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted and saved successfully for ResNet101! (Shape: (2100, 32))")


179648224/179648224 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
✅ Features extracted and saved successfully for ResNet101! (Shape: (2100, 32))


In [ ]:
print(conv5_features_101.shape)
print(fc6_features_101.shape)
print(conv5_reduced_101.shape)
print(fc6_reduced_101.shape)

(2100, 100352)
(2100, 2048)
(2100, 32)
(2100, 32)


ResNet152

In [ ]:
 import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet152 model with pre-trained weights
base_model_152 = ResNet152(weights='imagenet', include_top=True)

# Extracting features from 'avg_pool'
fc6_layer_152 = tf.keras.Model(inputs=base_model_152.input,
                               outputs=base_model_152.get_layer('avg_pool').output)

# Function to extract features
def extract_features(model, img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
fc6_features_152, labels = [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                fc6_feat = extract_features(fc6_layer_152, img_path)
                fc6_features_152.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
fc6_features_152 = np.array(fc6_features_152)

# Save raw extracted features
np.save(os.path.join(save_dir, "resnet152_fc6_fc6_features.npy"), fc6_features_152)

# Apply PCA to reduce dimensions to (2100, 32)
pca_fc6_152 = PCA(n_components=32)
fc6_reduced_152 = pca_fc6_152.fit_transform(fc6_features_152)

# Save reduced features
np.save(os.path.join(save_dir, "resnet152_fc6_32_features.npy"), fc6_reduced_152)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted and saved successfully for ResNet152! (Shape: (2100, 32))")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import os

# Load saved features
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"

resnet50_conv5 = np.load(os.path.join(save_dir, "resnet50_conv5_32_features.npy"))
resnet50_fc6 = np.load(os.path.join(save_dir, "resnet50_fc6_32_features.npy"))

resnet101_conv5 = np.load(os.path.join(save_dir, "resnet101_conv5_32_features.npy"))
resnet101_fc6 = np.load(os.path.join(save_dir, "resnet101_fc6_32_features.npy"))

labels = np.load(os.path.join(save_dir, "labels.npy"))

# Feature Fusion (Concatenation)
final_features_conv5 = np.concatenate((resnet50_conv5, resnet101_conv5), axis=1)  # Shape: (2100, 64)
final_features_fc6 = np.concatenate((resnet50_fc6, resnet101_fc6), axis=1)  # Shape: (2100, 64)

# Save the combined features
np.save(os.path.join(save_dir, "combined_conv5_features.npy"), final_features_conv5)
np.save(os.path.join(save_dir, "combined_fc6_features.npy"), final_features_fc6)

print("✅ Combined features saved successfully! (Shape: (2100, 64))")


✅ Combined features saved successfully! (Shape: (2100, 64))


In [ ]:
print(final_features_conv5.shape)
print(final_features_fc6.shape)

(2100, 64)
(2100, 64)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/combined_fc6_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-
        # sorted_indices = np.delete(sorted_indices, np.where(sorted_indices == i))  # More robust
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for combine ResNet 50&101 : {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for combine ResNet 50&101 : 71.27%


ANMRR

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def compute_anmrr(labels, distance_matrix):
    num_images = len(labels)
    anmrr_values = []

    for i in range(num_images):
        # Sort indices based on distance (ascending order)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the sorted labels
        sorted_labels = labels[sorted_indices]

        # Identify relevant items
        relevant_items = (sorted_labels == labels[i])
        num_relevant = np.sum(relevant_items)

        if num_relevant == 0:
            anmrr_values.append(1)  # Worst case if no relevant images found
            continue

        # Compute rank positions of relevant items
        rank_positions = np.where(relevant_items)[0] + 1  # Convert to 1-based indexing

        # Compute Modified Retrieval Rank (MRR)
        mrr = np.sum(rank_positions) / num_relevant

        # Compute Normalized Modified Retrieval Rank (NMRR)
        # K = min(2 * num_relevant, num_images)  # Dynamic cut-off based on relevant items
        K = num_images  # Since we are considering all images
        nmrr = (mrr - 0.5 * (1 + num_relevant)) / (1.25 * K - 0.5 * (1 + num_relevant))

        anmrr_values.append(nmrr)

    # Mean of NMRR values
    anmrr = np.mean(anmrr_values)
    return anmrr


# Compute and print ANMRR for each model (all images)

features = np.load(f"/content/drive/MyDrive/NewONeFeature_IFK_Vectors/combined_fc6_features.npy")
labels = np.load(f"/content/drive/MyDrive/NewONeFeature_IFK_Vectors/labels.npy")

# Compute distance matrix efficiently using cdist (Euclidean distance)
distance_matrix = cdist(features, features, metric='euclidean')

# Compute ANMRR for all images
anmrr_score = compute_anmrr(labels, distance_matrix)
print(f"ANMRR for combine  ResNet 50&101: {anmrr_score:.4f}")


ANMRR for combine  ResNet 50&101: 0.0370


In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# Load concatenated features (64 dimensions)
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
combined_features = np.load(f"{save_dir}/combined_fc6_features.npy")  # Shape: (2100, 64)

# Apply PCA to reduce to 32 dimensions
pca = PCA(n_components=32)
combined_features_32 = pca.fit_transform(combined_features)

# Save the new 32-dimensional features
np.save(f"{save_dir}/combined_fc6_32_features.npy", combined_features_32)

print(f"✅ Combined features reduced to 32 dimensions! New shape: {combined_features_32.shape}")


✅ Combined features reduced to 32 dimensions! New shape: (2100, 32)


In [ ]:
print(combined_features_32.shape)

(2100, 32)


MAP 32

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
features = np.load(f"{save_dir}/combined_fc6_32_features.npy")
labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-
        # sorted_indices = np.delete(sorted_indices, np.where(sorted_indices == i))  # More robust
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for combine ResNet 50&101 : {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for combine ResNet 50&101 : 72.83%


ANMRR 32

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def compute_anmrr(labels, distance_matrix):
    num_images = len(labels)
    anmrr_values = []

    for i in range(num_images):
        # Sort indices based on distance (ascending order)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the sorted labels
        sorted_labels = labels[sorted_indices]

        # Identify relevant items
        relevant_items = (sorted_labels == labels[i])
        num_relevant = np.sum(relevant_items)

        if num_relevant == 0:
            anmrr_values.append(1)  # Worst case if no relevant images found
            continue

        # Compute rank positions of relevant items
        rank_positions = np.where(relevant_items)[0] + 1  # Convert to 1-based indexing

        # Compute Modified Retrieval Rank (MRR)
        mrr = np.sum(rank_positions) / num_relevant

        # Compute Normalized Modified Retrieval Rank (NMRR)
        # K = min(2 * num_relevant, num_images)  # Dynamic cut-off based on relevant items
        K = num_images  # Since we are considering all images
        nmrr = (mrr - 0.5 * (1 + num_relevant)) / (1.25 * K - 0.5 * (1 + num_relevant))

        anmrr_values.append(nmrr)

    # Mean of NMRR values
    anmrr = np.mean(anmrr_values)
    return anmrr


# Compute and print ANMRR for each model (all images)

features = np.load(f"/content/drive/MyDrive/NewONeFeature_IFK_Vectors/combined_fc6_32_features.npy")
labels = np.load(f"/content/drive/MyDrive/NewONeFeature_IFK_Vectors/labels.npy")

# Compute distance matrix efficiently using cdist (Euclidean distance)
distance_matrix = cdist(features, features, metric='euclidean')

# Compute ANMRR for all images
anmrr_score = compute_anmrr(labels, distance_matrix)
print(f"ANMRR for combine  ResNet 50&101 : {anmrr_score:.4f}")


ANMRR for combine  ResNet 50&101 : 0.0310


clahe code on ResNet 101

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'conv5_block3_out' (Last convolutional block)
conv5_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                                 outputs=base_model_101.get_layer('conv5_block3_out').output)

# Extracting features from 'avg_pool'
fc6_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

# Function to Apply CLAHE
def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # Convert to LAB space
    l, a, b = cv2.split(lab)  # Split channels
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))  # CLAHE on L channel
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))  # Merge back
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)  # Convert back to BGR

# Function to Extract Features
def extract_features(model, img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)  # Read image
    img = cv2.resize(img, target_size)  # Resize to model input size
    img = apply_clahe(img)  # Apply CLAHE preprocessing

    # Convert to Keras-compatible format
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract Features
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
conv5_features_101, fc6_features_101, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_feat = extract_features(conv5_layer_101, img_path)
                fc6_feat = extract_features(fc6_layer_101, img_path)
                conv5_features_101.append(conv5_feat)
                fc6_features_101.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
conv5_features_101 = np.array(conv5_features_101)
fc6_features_101 = np.array(fc6_features_101)

# Save full-dimension features
np.save(os.path.join(save_dir, "resnet101_conv5_clahe_features.npy"), conv5_features_101)
np.save(os.path.join(save_dir, "resnet101_fc6_clahe_features.npy"), fc6_features_101)

# Apply PCA to reduce dimensions to (2100, 32)
pca_conv5_101 = PCA(n_components=32)
conv5_reduced_101 = pca_conv5_101.fit_transform(conv5_features_101)

pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save PCA-reduced features
np.save(os.path.join(save_dir, "resnet101_conv5_32_clahe_features.npy"), conv5_reduced_101)
np.save(os.path.join(save_dir, "resnet101_fc6_32_clahe_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted with CLAHE and saved successfully for ResNet101! (Shape: (2100, 32))")


✅ Features extracted with CLAHE and saved successfully for ResNet101! (Shape: (2100, 32))


ResNet50 clahe

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=True)

# Extracting features from 'conv5_block3_out' (Last convolutional block)
conv5_layer = tf.keras.Model(inputs=base_model.input,
                             outputs=base_model.get_layer('conv5_block3_out').output)

# Extracting features from 'avg_pool'
fc6_layer = tf.keras.Model(inputs=base_model.input,
                           outputs=base_model.get_layer('avg_pool').output)

# Function to Apply CLAHE
def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # Convert to LAB space
    l, a, b = cv2.split(lab)  # Split channels
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))  # CLAHE on L channel
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))  # Merge back
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)  # Convert back to BGR

# Function to Extract Features
def extract_features(model, img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)  # Read image
    img = cv2.resize(img, target_size)  # Resize to model input size
    img = apply_clahe(img)  # Apply CLAHE preprocessing

    # Convert to Keras-compatible format
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract Features
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
conv5_features, fc6_features, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                conv5_feat = extract_features(conv5_layer, img_path)
                fc6_feat = extract_features(fc6_layer, img_path)
                conv5_features.append(conv5_feat)
                fc6_features.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
conv5_features = np.array(conv5_features)
fc6_features = np.array(fc6_features)

# Apply PCA to reduce dimensions to (2100, 32)
pca_conv5 = PCA(n_components=32)
conv5_reduced = pca_conv5.fit_transform(conv5_features)

pca_fc6 = PCA(n_components=32)
fc6_reduced = pca_fc6.fit_transform(fc6_features)

# Save features as separate .npy files
np.save(os.path.join(save_dir, "resnet50_conv5_32_clahe_features.npy"), conv5_reduced)
np.save(os.path.join(save_dir, "resnet50_fc6_32_clahe_features.npy"), fc6_reduced)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ Features extracted with CLAHE and saved successfully for ResNet50! (Shape: (2100, 32))")


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
✅ Features extracted with CLAHE and saved successfully for ResNet50! (Shape: (2100, 32))


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
# features = np.load(f"{save_dir}/resnet50_fc6_32_clahe_features.npy")
features = np.load(f"{save_dir}/resnet101_fc6_32_clahe_features.npy")

labels = np.load(f"{save_dir}/labels.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-
        # sorted_indices = np.delete(sorted_indices, np.where(sorted_indices == i))  # More robust
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for combine ResNet 50&101 : {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for combine ResNet 50&101 : 58.93%
